In [1]:
from pathlib import Path
import re
import warnings
import numpy as np
import pandas as pd
import pycountry

# Silence noisy openpyxl style warning from WHO workbooks.
warnings.filterwarnings(
    "ignore",
    message="Workbook contains no default style, apply openpyxl's default",
    category=UserWarning,
)

# Paths relative to this notebook's location (Codes/)
DATA_DIR = Path("../Assets/full_data")
OUT_DIR = Path("../Assets/cleaner_exports")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "life_expectancy": DATA_DIR / "Life_expectancy.xlsx",
    "adult_mortality": DATA_DIR / "Adult_mortality.xlsx",
    "alcohol": DATA_DIR / "alcohol_consumption.xlsx",
    "che": DATA_DIR / "health_expenditure.xlsx",
    "hepb3": DATA_DIR / "HepB3.xlsx",
    "infant_deaths": DATA_DIR / "Infant_deaths.xlsx",
    "overweight": DATA_DIR / "overweight_adults.xlsx",
    "polio": DATA_DIR / "Polio.xlsx",
    "u5": DATA_DIR / "U5_mortality.xlsx",
    "underweight": DATA_DIR / "underweight_adults.xlsx",
    "dtp3": DATA_DIR / "Diptheria.xlsx",
    "hiv": DATA_DIR / "HIV.xlsx",
    "gdp": DATA_DIR / "GDP_and_population.csv",

    "thinness": DATA_DIR / "Thinness.xlsx",
}

missing = [k for k, p in FILES.items() if not p.exists()]
if missing:
    raise FileNotFoundError(
        "Missing files\n"
        + "\n".join([f"- {k}: {FILES[k]}" for k in missing])
        + "\n\nFix DATA_DIR to your local folder path."
    )

YEARS = tuple(range(2000, 2022))  # 2000–2021 inclusive

print("All files found.")
print("DATA_DIR =", DATA_DIR.resolve())
print("OUT_DIR  =", OUT_DIR.resolve())
print("Year range:", YEARS[0], "-", YEARS[-1])


All files found.
DATA_DIR = /Users/bikki/Documents/Projects_Spring_2026/STATS201/Assets/full_data
OUT_DIR  = /Users/bikki/Documents/Projects_Spring_2026/STATS201/Assets/cleaner_exports
Year range: 2000 - 2021


In [2]:
# -----------------------------
# Country name normalization
# -----------------------------
SPECIAL_COUNTRY_MAP = {
    # WHO-style names -> pycountry-friendly names
    "Bolivia (Plurinational State of)": "Bolivia",
    "Venezuela (Bolivarian Republic of)": "Venezuela",
    "Iran (Islamic Republic of)": "Iran, Islamic Republic of",
    "United Republic of Tanzania": "Tanzania, United Republic of",
    "Republic of Moldova": "Moldova, Republic of",
    "Micronesia (Federated States of)": "Micronesia, Federated States of",
    "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
    "United States of America": "United States",
    "Netherlands (Kingdom of the)": "Netherlands",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Côte d’Ivoire": "Côte d'Ivoire",
    "occupied Palestinian territory, including east Jerusalem": "Palestine, State of",
    "State of Palestine": "Palestine, State of",
    "Democratic Republic of the Congo": "Congo, The Democratic Republic of the",
    "Holy See": "Holy See (Vatican City State)",
    "Vatican": "Holy See (Vatican City State)",
    "The former Yugoslav Republic of Macedonia": "North Macedonia",

    # Korea naming (avoid wrong fuzzy matches)
    "Republic of Korea": "Korea, Republic of",
    "South Korea": "Korea, Republic of",
    "North Korea": "Korea, Democratic People's Republic of",

    # Türkiye spelling
    "Türkiye": "Turkey",

    # Kosovo variants
    "Kosovo (in accordance with UN Security Council resolution 1244 (1999))": "Kosovo",
    "Kosovo": "Kosovo",
}

REGION_KEYWORDS = {
    # used only as a secondary filter if needed
    "africa", "asia", "europe", "world", "global", "income", "high income", "low income",
    "middle income", "upper middle", "lower middle", "euro area", "oecd", "arab",
    "sub-saharan", "latin america", "caribbean", "north america", "pacific",
}

def normalize_text(x):
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return None
    s = str(x).strip()
    s = re.sub(r"\s+", " ", s)
    return s

def normalize_country_name(name):
    name = normalize_text(name)
    if not name:
        return None
    if name.lower().startswith("applied filters"):
        return None
    return SPECIAL_COUNTRY_MAP.get(name, name)

def iso3_to_canonical_name(iso3: str) -> str:
    if iso3 == "XKX":
        return "Kosovo"
    c = pycountry.countries.get(alpha_3=iso3)
    return c.name if c else iso3

def country_to_iso3(name):
    name = normalize_country_name(name)
    if not name:
        return None
    if name == "Kosovo":
        return "XKX"
    try:
        return pycountry.countries.lookup(name).alpha_3
    except Exception:
        return None

def parse_point_estimate(x):
    """
    Parses WHO cells like:
      '59.1 [58.3-59.9]' -> 59.1
      '76 646 [61 256-94 911]' -> 76646
      '<0.1 [<0.1 - 0.3]' -> 0.1
      '..' -> NaN
    """
    if x is None:
        return np.nan
    if isinstance(x, (int, float, np.integer, np.floating)) and not (isinstance(x, float) and np.isnan(x)):
        return float(x)

    s = str(x).strip()
    if s == "" or s in {"..", "...", "NA", "N/A"}:
        return np.nan
    if s.lower().startswith("applied filters"):
        return np.nan

    # Keep only the point estimate before bracket
    if "[" in s:
        s = s.split("[", 1)[0].strip()

    # Remove < or >
    if s.startswith("<") or s.startswith(">"):
        s = s[1:].strip()

    # Remove separators
    s = s.replace(",", "").replace("\u00a0", " ").replace(" ", "")

    m = re.search(r"-?\d+(\.\d+)?", s)
    if not m:
        return np.nan
    return float(m.group(0))

In [3]:
def read_excel_raw(path, sheet_name=0):
    return pd.read_excel(path, sheet_name=sheet_name, header=None, engine="openpyxl")

def detect_row(raw: pd.DataFrame, label: str):
    col0 = raw.iloc[:, 0].astype(str).str.strip().str.lower()
    hits = raw.index[col0 == label.lower()]
    return int(hits[0]) if len(hits) else None

def parse_who_export_excel(
    path,
    *,
    indicator_contains=None,
    years=range(2000, 2022),
    dim_filters=None,
    sheet_name=0,
    value_name="Value",
    allow_multiple_matches=False,
):
    """
    Returns tidy DF:
      ISO3, Country, Year, <value_name>

    Supports:
      - LONG layout: Location | Period | multiple value columns
      - WIDE layout: Location + year columns (via Period row)
    Filters:
      - indicator_contains: substring match on 'Indicator' row (if present)
      - dim_filters: e.g. {"Dim1":"Both sexes", "Dim2":"18+  years"}
    """
    years = set(int(y) for y in years)
    dim_filters = {k: normalize_text(v) for k, v in (dim_filters or {}).items()}

    raw = read_excel_raw(path, sheet_name=sheet_name)
    raw = raw.dropna(axis=0, how="all").dropna(axis=1, how="all").reset_index(drop=True)

    indicator_idx = detect_row(raw, "Indicator")
    period_idx = detect_row(raw, "Period")
    location_idx = detect_row(raw, "Location")

    # Detect LONG layout: Location row has "Period" in col 1
    is_long = False
    if location_idx is not None and raw.shape[1] >= 2:
        if str(raw.iloc[location_idx, 1]).strip().lower() == "period":
            is_long = True

    # Dim rows (Dim1, Dim2, ...)
    dim_row_idxs = {
        str(raw.iloc[i, 0]).strip(): i
        for i in range(raw.shape[0])
        if re.fullmatch(r"Dim\d+", str(raw.iloc[i, 0]).strip())
    }

    if is_long:
        header_idx = location_idx
        value_cols = list(range(2, raw.shape[1]))  # data columns after Location, Period

        selected_cols = []
        for j in value_cols:
            ok = True

            if indicator_contains and indicator_idx is not None:
                ind = normalize_text(raw.iloc[indicator_idx, j])
                if not ind or indicator_contains.lower() not in ind.lower():
                    ok = False

            for dim, wanted in dim_filters.items():
                if dim in dim_row_idxs:
                    dim_val = normalize_text(raw.iloc[dim_row_idxs[dim], j])
                    if dim_val != wanted:
                        ok = False

            if ok:
                selected_cols.append(j)

        if not selected_cols:
            raise ValueError(f"No matching columns found in LONG layout: {path.name}")

        if (not allow_multiple_matches) and len(selected_cols) != 1:
            raise ValueError(
                f"Matched {len(selected_cols)} columns in {path.name} (LONG). "
                f"Make filters stricter (indicator_contains/dim_filters)."
            )

        df = raw.iloc[header_idx + 1 :, [0, 1] + selected_cols].copy()

        if len(selected_cols) == 1:
            df.columns = ["Country", "Year", value_name]
            value_cols_out = [value_name]
        else:
            out_cols = [f"{value_name}_{i}" for i in range(len(selected_cols))]
            df.columns = ["Country", "Year"] + out_cols
            value_cols_out = out_cols

        df["Country"] = df["Country"].apply(normalize_country_name)
        df = df[df["Country"].notna()]

        df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
        df = df[df["Year"].isin(list(years))]

        for c in value_cols_out:
            df[c] = df[c].apply(parse_point_estimate)

        df["ISO3"] = df["Country"].apply(country_to_iso3)
        df = df[df["ISO3"].notna()]
        df["Country"] = df["ISO3"].apply(iso3_to_canonical_name)

        # If multiple, keep first non-null across matches
        if len(value_cols_out) > 1:
            df[value_name] = df[value_cols_out].bfill(axis=1).iloc[:, 0]
            df = df.drop(columns=value_cols_out)

        return df[["ISO3", "Country", "Year", value_name]].reset_index(drop=True)

    # WIDE layout
    if location_idx is None or period_idx is None:
        raise ValueError(f"Could not detect WIDE layout rows (Location/Period): {path.name}")

    value_cols = list(range(1, raw.shape[1]))
    years_arr = [normalize_text(raw.iloc[period_idx, j]) for j in value_cols]
    ind_arr = [
        normalize_text(raw.iloc[indicator_idx, j]) if indicator_idx is not None else None
        for j in value_cols
    ]
    dim_arrs = {
        dim: [normalize_text(raw.iloc[idx, j]) for j in value_cols]
        for dim, idx in dim_row_idxs.items()
    }

    selected_cols = []
    selected_years = []
    for idx, j in enumerate(value_cols):
        y = years_arr[idx]
        if y is None:
            continue
        try:
            y_int = int(float(y))
        except Exception:
            continue
        if y_int not in years:
            continue

        ok = True

        if indicator_contains and indicator_idx is not None:
            ind = ind_arr[idx]
            if not ind or indicator_contains.lower() not in ind.lower():
                ok = False

        for dim, wanted in dim_filters.items():
            if dim in dim_arrs:
                if dim_arrs[dim][idx] != wanted:
                    ok = False

        if ok:
            selected_cols.append(j)
            selected_years.append(y_int)

    if not selected_cols:
        raise ValueError(f"No matching year/dim columns found in WIDE layout: {path.name}")

    data = raw.iloc[location_idx + 1 :, [0] + selected_cols].copy()
    data.columns = ["Country"] + [str(y) for y in selected_years]

    data["Country"] = data["Country"].apply(normalize_country_name)
    data = data[data["Country"].notna()]
    data = data[~data["Country"].str.lower().str.startswith("applied filters")]

    df = data.melt(id_vars="Country", var_name="Year", value_name=value_name)
    df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
    df[value_name] = df[value_name].apply(parse_point_estimate)

    df["ISO3"] = df["Country"].apply(country_to_iso3)
    df = df[df["ISO3"].notna()]
    df["Country"] = df["ISO3"].apply(iso3_to_canonical_name)

    return df[["ISO3", "Country", "Year", value_name]].reset_index(drop=True)

In [4]:
life_expectancy = parse_who_export_excel(
    FILES["life_expectancy"],
    indicator_contains="Life expectancy at birth",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes"},
    value_name="life_expectancy_at_birth_both",
)

adult_mortality = parse_who_export_excel(
    FILES["adult_mortality"],
    indicator_contains="Adult mortality rate",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes"},
    value_name="adult_mortality_rate_15_60_both",
)

alcohol = parse_who_export_excel(
    FILES["alcohol"],
    indicator_contains="Alcohol, total per capita",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes"},
    value_name="alcohol_per_capita_15plus_litres",
)

che = parse_who_export_excel(
    FILES["che"],
    indicator_contains="Current health expenditure",
    years=YEARS,
    value_name="che_pct_gdp",
)

hepb3 = parse_who_export_excel(
    FILES["hepb3"],
    indicator_contains="Hepatitis B",
    years=YEARS,
    value_name="hepb3_coverage_pct",
)

infant_deaths = parse_who_export_excel(
    FILES["infant_deaths"],
    indicator_contains="Number of infant deaths",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes"},
    value_name="infant_deaths_both",
)

overweight = parse_who_export_excel(
    FILES["overweight"],
    indicator_contains="Prevalence of overweight among adults",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes", "Dim2": "18+  years"},
    value_name="overweight_adults_bmi_ge_25_pct",
)

polio = parse_who_export_excel(
    FILES["polio"],
    indicator_contains="Polio (Pol3)",
    years=YEARS,
    value_name="pol3_coverage_pct",
)

u5 = parse_who_export_excel(
    FILES["u5"],
    indicator_contains="Under-five mortality rate",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes"},
    value_name="u5_mortality_rate_per_1000",
)

underweight = parse_who_export_excel(
    FILES["underweight"],
    indicator_contains="Prevalence of underweight among adults",
    years=YEARS,
    dim_filters={"Dim1": "Both sexes", "Dim2": "18+  years"},
    value_name="underweight_adults_bmi_lt_18_pct",
)

dtp3 = parse_who_export_excel(
    FILES["dtp3"],
    indicator_contains="DTP3",
    years=YEARS,
    value_name="dtp3_coverage_pct",
)

hiv = parse_who_export_excel(
    FILES["hiv"],
    indicator_contains="Prevalence of HIV among adults",
    years=YEARS,
    value_name="hiv_prev_15_49_pct",
)

thinness = parse_who_export_excel(

    FILES["thinness"],

    indicator_contains="Prevalence of thinness among children and adolescents",

    years=YEARS,

    dim_filters={"Dim1": "Both sexes", "Dim2": "5-19  years"},

    value_name="thinness_children_adolescents_bmi_lt_minus2_pct",

)



WHO_TABLES = {
    "life_expectancy": life_expectancy,
    "adult_mortality": adult_mortality,
    "alcohol": alcohol,
    "che": che,
    "hepb3": hepb3,
    "infant_deaths": infant_deaths,
    "overweight": overweight,
    "polio": polio,
    "u5": u5,
    "underweight": underweight,
    "dtp3": dtp3,
    "hiv": hiv,

    "thinness": thinness,
}

# Ensure no duplicates per ISO3-Year
for k, df in WHO_TABLES.items():
    dups = df.duplicated(["ISO3", "Year"]).sum()
    if dups:
        raise ValueError(f"Duplicate ISO3-Year rows in '{k}': {dups}")

print("WHO indicators extracted.")
print({k: v.shape for k, v in WHO_TABLES.items()})

WHO indicators extracted.
{'life_expectancy': (4048, 4), 'adult_mortality': (4048, 4), 'alcohol': (4114, 4), 'che': (4268, 4), 'hepb3': (4246, 4), 'infant_deaths': (4378, 4), 'overweight': (4356, 4), 'polio': (4246, 4), 'u5': (4378, 4), 'underweight': (4356, 4), 'dtp3': (4246, 4), 'hiv': (1158, 4), 'thinness': (4356, 4)}


In [5]:
def parse_worldbank_gdp_per_capita(path, years=range(2000, 2022)):
    df = pd.read_csv(path)

    # Filter target series
    if "Series Code" not in df.columns:
        raise ValueError("GDP CSV does not have 'Series Code' column; layout unexpected.")

    df = df[df["Series Code"].eq("NY.GDP.PCAP.KD")].copy()

    # Select year columns that exist
    year_cols = []
    for y in years:
        col = f"{y} [YR{y}]"
        if col in df.columns:
            year_cols.append(col)

    keep_cols = ["Country Name", "Country Code"] + year_cols
    df = df[keep_cols].copy()

    long = df.melt(
        id_vars=["Country Name", "Country Code"],
        var_name="Year",
        value_name="gdp_per_capita_const_2015_usd",
    )

    long["Year"] = long["Year"].str.extract(r"(\d{4})").astype("Int64")
    long["gdp_per_capita_const_2015_usd"] = pd.to_numeric(
        long["gdp_per_capita_const_2015_usd"].replace("..", np.nan),
        errors="coerce",
    )

    long = long.rename(columns={"Country Code": "ISO3", "Country Name": "Country"})

    # Country-only filter: keep real ISO3 + XKX, drop regions/aggregates
    iso3_set = {c.alpha_3 for c in pycountry.countries}
    long = long[long["ISO3"].isin(iso3_set.union({"XKX"}))].copy()

    long["Country"] = long["ISO3"].apply(iso3_to_canonical_name)

    return long[["ISO3", "Country", "Year", "gdp_per_capita_const_2015_usd"]].reset_index(drop=True)

gdp_pc = parse_worldbank_gdp_per_capita(FILES["gdp"], years=YEARS)
print("GDP per capita extracted:", gdp_pc.shape)

GDP per capita extracted: (4752, 4)


In [6]:
# Use the union of ISO3 from WHO tables (already country-only)
all_iso3 = set()
for df in WHO_TABLES.values():
    all_iso3.update(df["ISO3"].dropna().unique().tolist())

panel = (
    pd.MultiIndex.from_product([sorted(all_iso3), YEARS], names=["ISO3", "Year"])
      .to_frame(index=False)
)

panel["Country"] = panel["ISO3"].apply(iso3_to_canonical_name)
panel = panel[["Country", "ISO3", "Year"]].copy()

# Merge each WHO indicator
for k, df in WHO_TABLES.items():
    value_cols = [c for c in df.columns if c not in {"ISO3", "Country", "Year"}]
    if len(value_cols) != 1:
        raise ValueError(f"Expected 1 value col in {k}, got {value_cols}")
    vcol = value_cols[0]
    panel = panel.merge(df[["ISO3", "Year", vcol]], on=["ISO3", "Year"], how="left")

# Merge GDP per capita
panel = panel.merge(
    gdp_pc[["ISO3", "Year", "gdp_per_capita_const_2015_usd"]],
    on=["ISO3", "Year"],
    how="left",
)

# Safety checks
if panel.duplicated(["ISO3", "Year"]).sum() != 0:
    raise ValueError("Duplicate ISO3-Year rows found in final panel (should never happen).")

print("Final panel shape:", panel.shape)
print("Countries:", panel["ISO3"].nunique())
print("Years:", int(panel["Year"].min()), "-", int(panel["Year"].max()))

panel.head(10)

Final panel shape: (4532, 17)
Countries: 206
Years: 2000 - 2021


,Country,ISO3,Year,life_expectancy_at_birth_both,adult_mortality_rate_15_60_both,alcohol_per_capita_15plus_litres,che_pct_gdp,hepb3_coverage_pct,infant_deaths_both,overweight_adults_bmi_ge_25_pct,pol3_coverage_pct,u5_mortality_rate_per_1000,underweight_adults_bmi_lt_18_pct,dtp3_coverage_pct,hiv_prev_15_49_pct,thinness_children_adolescents_bmi_lt_minus2_pct,gdp_per_capita_const_2015_usd
0,Afghanistan,AFG,2000,53.8,378.0,0.0,NaN,0.0,110740.0,17.9,24.0,131.7,20.4,24.0,0.1,15.9,308.318270
1,Afghanistan,AFG,2001,53.9,380.0,0.0,NaN,0.0,107225.0,18.9,35.0,127.4,19.6,33.0,NaN,15.5,277.118051
2,Afghanistan,AFG,2002,55.2,367.0,0.0,9.44,0.0,103124.0,19.9,36.0,123.1,18.9,36.0,NaN,15.1,338.139974
3,Afghanistan,AFG,2003,56.1,353.0,0.0,8.94,0.0,103127.0,21.0,41.0,118.7,18.1,41.0,NaN,14.8,346.071627
4,Afghanistan,AFG,2004,56.5,346.0,0.0,9.81,0.0,103462.0,22.2,50.0,114.2,17.3,50.0,NaN,14.4,338.637274
5,Afghanistan,AFG,2005,56.8,343.0,0.0,9.95,0.0,101026.0,23.3,58.0,109.7,16.5,58.0,0.1,14.0,363.640141
6,Afghanistan,AFG,2006,57.0,342.0,0.0,10.62,0.0,98399.0,24.5,58.0,105.1,15.7,58.0,NaN,13.6,367.758312
7,Afghanistan,AFG,2007,57.3,339.0,0.0,9.90,63.0,96353.0,25.8,63.0,100.7,15.0,63.0,NaN,13.3,410.757729
8,Afghanistan,AFG,2008,57.9,329.0,0.0,10.26,64.0,92841.0,27.1,64.0,96.3,14.3,64.0,NaN,12.9,417.647283
9,Afghanistan,AFG,2009,58.5,320.0,0.0,9.82,63.0,90160.0,28.4,63.0,92.2,13.5,63.0,NaN,12.5,488.830652


In [7]:
RENAME_TO_REQUESTED = {
    "life_expectancy_at_birth_both": "Life expectancy at birth (years) Both sexes",
    "adult_mortality_rate_15_60_both": "Adult mortality rate (probability of dying between 15 and 60 years per 1000 population) Both sexes",
    "alcohol_per_capita_15plus_litres": "Alcohol, total per capita (15+) consumption (in litres of pure alcohol) (SDG Indicator 3.5.2), three-year average Both sexes",
    "che_pct_gdp": "Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)",
    "hepb3_coverage_pct": "Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)",
    "infant_deaths_both": "Number of infant deaths Both sexes",
    "overweight_adults_bmi_ge_25_pct": "Prevalence of overweight among adults, BMI ≥ 25 (age-standardized estimate) (%) Both sexes",
    "pol3_coverage_pct": "Polio (Pol3) immunization coverage among 1-year-olds (%)",
    "u5_mortality_rate_per_1000": "Under-five mortality rate (probability of dying by age 5 per 1000 live births) Both sexes",
    "underweight_adults_bmi_lt_18_pct": "Prevalence of underweight among adults, BMI < 18 (age-standardized estimate) (%) Both sexes",
    "dtp3_coverage_pct": "Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)",
    "hiv_prev_15_49_pct": "Prevalence of HIV among adults aged 15 to 49 (%)",

    "thinness_children_adolescents_bmi_lt_minus2_pct": "Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%) Both sexes 5-19 years",
    "gdp_per_capita_const_2015_usd": "GDP per capita (constant 2015 US$)",
}

panel_requested = panel.rename(columns=RENAME_TO_REQUESTED).copy()

final_cols = [
    "Country",
    "Year",
    "Life expectancy at birth (years) Both sexes",
    "Adult mortality rate (probability of dying between 15 and 60 years per 1000 population) Both sexes",
    "Alcohol, total per capita (15+) consumption (in litres of pure alcohol) (SDG Indicator 3.5.2), three-year average Both sexes",
    "Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)",
    "Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)",
    "Number of infant deaths Both sexes",
    "Prevalence of overweight among adults, BMI ≥ 25 (age-standardized estimate) (%) Both sexes",
    "Polio (Pol3) immunization coverage among 1-year-olds (%)",
    "Under-five mortality rate (probability of dying by age 5 per 1000 live births) Both sexes",
    "Prevalence of underweight among adults, BMI < 18 (age-standardized estimate) (%) Both sexes",
    "Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)",
    "Prevalence of HIV among adults aged 15 to 49 (%)",

    "Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%) Both sexes 5-19 years",
    "GDP per capita (constant 2015 US$)",
]

# Keep stable column order including ISO3.
panel_requested = panel_requested[["Country", "ISO3", "Year"] + final_cols[2:]].copy()

# Optional: infant deaths as integer-like (nullable)
panel_requested["Number of infant deaths Both sexes"] = (
    panel_requested["Number of infant deaths Both sexes"].round().astype("Int64")
)

panel_requested.head(10)


,Country,ISO3,Year,Life expectancy at birth (years) Both sexes,Adult mortality rate (probability of dying between 15 and 60 years per 1000 population) Both sexes,"Alcohol, total per capita (15+) consumption (in litres of pure alcohol) (SDG Indicator 3.5.2), three-year average Both sexes",Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%),Hepatitis B (HepB3) immunization coverage among 1-year-olds (%),Number of infant deaths Both sexes,"Prevalence of overweight among adults, BMI ≥ 25 (age-standardized estimate) (%) Both sexes",Polio (Pol3) immunization coverage among 1-year-olds (%),Under-five mortality rate (probability of dying by age 5 per 1000 live births) Both sexes,"Prevalence of underweight among adults, BMI < 18 (age-standardized estimate) (%) Both sexes",Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%),Prevalence of HIV among adults aged 15 to 49 (%),"Prevalence of thinness among children and adolescents, BMI < -2 standard deviations below the median (crude estimate) (%) Both sexes 5-19 years",GDP per capita (constant 2015 US$)
0,Afghanistan,AFG,2000,53.8,378.0,0.0,NaN,0.0,110740,17.9,24.0,131.7,20.4,24.0,0.1,15.9,308.318270
1,Afghanistan,AFG,2001,53.9,380.0,0.0,NaN,0.0,107225,18.9,35.0,127.4,19.6,33.0,NaN,15.5,277.118051
2,Afghanistan,AFG,2002,55.2,367.0,0.0,9.44,0.0,103124,19.9,36.0,123.1,18.9,36.0,NaN,15.1,338.139974
3,Afghanistan,AFG,2003,56.1,353.0,0.0,8.94,0.0,103127,21.0,41.0,118.7,18.1,41.0,NaN,14.8,346.071627
4,Afghanistan,AFG,2004,56.5,346.0,0.0,9.81,0.0,103462,22.2,50.0,114.2,17.3,50.0,NaN,14.4,338.637274
5,Afghanistan,AFG,2005,56.8,343.0,0.0,9.95,0.0,101026,23.3,58.0,109.7,16.5,58.0,0.1,14.0,363.640141
6,Afghanistan,AFG,2006,57.0,342.0,0.0,10.62,0.0,98399,24.5,58.0,105.1,15.7,58.0,NaN,13.6,367.758312
7,Afghanistan,AFG,2007,57.3,339.0,0.0,9.90,63.0,96353,25.8,63.0,100.7,15.0,63.0,NaN,13.3,410.757729
8,Afghanistan,AFG,2008,57.9,329.0,0.0,10.26,64.0,92841,27.1,64.0,96.3,14.3,64.0,NaN,12.9,417.647283
9,Afghanistan,AFG,2009,58.5,320.0,0.0,9.82,63.0,90160,28.4,63.0,92.2,13.5,63.0,NaN,12.5,488.830652


In [8]:
# Missingness % by column
missing_pct = (panel_requested.isna().mean() * 100).sort_values(ascending=False)
print("Missing % by column:")
print(missing_pct)

# Coverage by year (rows per year)
print("\nRows per year:")
print(panel_requested.groupby("Year").size())

# Export to cleaner_exports
out_path = OUT_DIR / "health_panel_2000_2021.csv"
panel_requested.to_csv(out_path, index=False)
print("\nSaved:", out_path.resolve())


Missing % by column:
Prevalence of HIV among adults aged 15 to 49 (%)                                                                                                   80.406002
Life expectancy at birth (years) Both sexes                                                                                                        10.679612
Adult mortality rate (probability of dying between 15 and 60 years per 1000 population) Both sexes                                                 10.679612
Alcohol, total per capita (15+) consumption (in litres of pure alcohol) (SDG Indicator 3.5.2), three-year average Both sexes                        9.223301
Current health expenditure (CHE) as percentage of gross domestic product (GDP) (%)                                                                  7.811121
Hepatitis B (HepB3) immunization coverage among 1-year-olds (%)                                                                                     6.729921
Polio (Pol3) immunization coverage am